In [282]:
import numpy as np
import pandas as pd
import pymorphy2
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
morph = pymorphy2.MorphAnalyzer()
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [252]:
%%time
for el in range(len(train_df['title'])):
    words = []
    for w in train_df['title'][el].split():
        w = w.rstrip(string.punctuation)
        w = w.lower()
        w = lemmatizer.lemmatize(w)
        w = re.sub('[^A-Za-zА-Яа-я]+','',  w)
        if len(w) == 0:
            continue
        p = morph.parse(w)[0]
        words.append(p.normal_form)
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'} 
    train_df['title'][el] = ' '.join([word for word in words if pos(word) not in functors_pos])
    words = nltk.word_tokenize(train_df['title'][el])
    without_w = [word for word in words if not word in stop_words]
    #print(without_w)
    train_df['title'][el] = ' '.join(without_w)
train_df

<timed exec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 15min 14s, sys: 657 ms, total: 15min 15s
Wall time: 15min 15s


,id,url,title,target
0,0,m.kp.md,эксминистр экономика молдова глава мидэя цель ...,False
1,1,www.kp.by,этот песня стать известный многий телезритель ...,False
2,2,fanserials.tv,банша сезон серия бремя красота смотреть онлайн,False
3,3,colorbox.spb.ru,бесить я картинка,False
4,4,tula-sport.ru,новомосковск сыграть следжхоккеист алексинский...,False
...,...,...,...,...
135304,135304,mail.ru,пора тюльпан турецкий сериал русский язык резу...,False
135305,135305,www.ntv.ru,остросюжетный сериал шеф игра повышение я серия,False
135306,135306,topclassiccarsforsale.com,plymouth special deluxe hot rod automaticsmall...,False
135307,135307,wowcream.ru,купить skin сыворотка питательный power formul...,False


In [283]:
for el in range(len(train_df['url'])):
    if 'www.' in train_df['url'][el]:
        train_df['url'][el] = train_df['url'][el][4:]
    pos_point = train_df['url'][el].rfind('.')
    train_df['url'][el] = train_df['url'][el][:pos_point]
    train_df['url'][el] = re.sub('[^A-Za-z1-10]+','',  train_df['url'][el])
train_df

<ipython-input-283-52598dae4f3d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['url'][el] = train_df['url'][el][:pos_point]
<ipython-input-283-52598dae4f3d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['url'][el] = re.sub('[^A-Za-z1-10]+','',  train_df['url'][el])
<ipython-input-283-52598dae4f3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['url'][el] = train_df['url'][el][4:]


,id,url,title,target
0,0,mkp,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,kp,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorboxspb,Не Беси Меня Картинки,False
4,4,tulasport,В Новомосковске сыграют следж-хоккеисты алекси...,False
...,...,...,...,...
135304,135304,mail,пора тюльпанов турецкий сериал на русском язык...,False
135305,135305,ntv,Остросюжетный сериал «Шеф. Игра на повышение»....,False
135306,135306,topclassiccarsforsale,"1941 Plymouth Special Deluxe Hot Rod, Automati...",False
135307,135307,wowcream,Купить It's Skin Сыворотка питательная Power 1...,False


In [284]:
from sklearn.metrics import f1_score

In [285]:
X_train = train_df["title"].values
X_test = test_df["title"].values
Xu_train = train_df["url"].values
Xu_test = test_df["url"].values
y_train = train_df["target"].astype(int).values


# Simple baseline

In [298]:
y_pred = [int("порно" in text) for text in X_train]

In [299]:
f1_score(y_train, y_pred)

0.31242758551206484

# ML baseline

In [289]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
vectorizeru = CountVectorizer()

model1 = MultinomialNB()
model2 = MultinomialNB()
modelu = MultinomialNB()

In [290]:
X1_train_vectorized = vectorizer1.fit_transform(X_train)
X2_train_vectorized = vectorizer2.fit_transform(X_train)
Xu_train_vectorized = vectorizeru.fit_transform(Xu_train)

In [291]:
X1_train_vectorized#xX1_train_vectorized

<135309x189852 sparse matrix of type '<class 'numpy.int64'>'
	with 1202659 stored elements in Compressed Sparse Row format>

In [292]:
print(X_train[0])    

Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�


In [293]:
%%time

model1.fit(
    X1_train_vectorized,
    y_train
)


model2.fit(
    X2_train_vectorized,
    y_train
)

modelu.fit(
    Xu_train_vectorized,
    y_train
)

CPU times: user 88.9 ms, sys: 10.4 ms, total: 99.2 ms
Wall time: 98.2 ms


MultinomialNB()

In [294]:
print(f1_score(y_test, yu_pred))
print(f1_score(y_test, y1_pred))
print(f1_score(y_test, y2_pred))

0.8709961798413165
0.9088775909268675
0.8775967717829921


### Submit

In [264]:
%%time
for el in range(len(test_df['title'])):
    words = []
    for w in test_df['title'][el].split():
        w = w.rstrip(string.punctuation)
        w = w.lower()
        w = lemmatizer.lemmatize(w)
        w = re.sub('[^A-Za-zА-Яа-я]+','',  w)
        if len(w) == 0:
            continue
        p = morph.parse(w)[0]
        words.append(p.normal_form)
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'} 
    test_df['title'][el] = ' '.join([word for word in words if pos(word) not in functors_pos])
    words = nltk.word_tokenize(test_df['title'][el])
    without_w = [word for word in words if not word in stop_words]
    test_df['title'][el] = ' '.join(without_w)
test_df

<timed exec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 21min 16s, sys: 747 ms, total: 21min 17s
Wall time: 21min 17s


,id,url,title
0,135309,www.kommersant.ru,шестой кассационный суд самара начать работа р...
1,135310,urexpert.online,такой индексация алименты кто какой случай про...
2,135311,imperimeha.ru,женщина империя мех part
3,135312,national-porn.com,небритый волосатый киска порно весь страна нац...
4,135313,2gis.ru,
...,...,...,...
165373,300682,etp.armtek.ru,armtek запчасть грузовой легковой автомобиль о...
165374,300683,mail.ru,лилия якупов караганда карагандинский область ...
165375,300684,xn----8sbnqchpeeeth.xn--p1ai,администрация лесной район тверской область я ...
165376,300685,www-sunhome-ru.cdn.ampproject.org,сонник изменение сознание сниться изменение со...


In [295]:
%%time
for el in range(len(test_df['url'])):
    if 'www.' in test_df['url'][el]:
        test_df['url'][el] = test_df['url'][el][4:]
    pos_point = test_df['url'][el].rfind('.')
    test_df['url'][el] = test_df['url'][el][:pos_point]
    test_df['url'][el] = re.sub('[^A-Za-z1-10]+','',  test_df['url'][el])
test_df

<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 3min 7s, sys: 1.58 s, total: 3min 8s
Wall time: 3min 7s


,id,url,title
0,135309,kommersant,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha,Женщинам | Империя Меха - Part 12
3,135312,nationalporn,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,gis,67
...,...,...,...
165373,300682,etparmtek,Armtek - запчасти для грузовых и легковых авто...
165374,300683,mail,"Лилия Якупова - Караганда, Карагандинская обла..."
165375,300684,xnsbnqchpeeeth,Администрация Лесного района Тверской области ...
165376,300685,wwwsunhomerucdnampproject,Сонник Изменение сознания. К чему снится Измен...


In [297]:
X1_test_vectorized = vectorizer1.transform(X_test)
X2_test_vectorized = vectorizer2.transform(X_test)
Xu_test_vectorized = vectorizeru.transform(Xu_test)


test_df["target1"] = model1.predict(X1_test_vectorized).astype(bool)
test_df["target2"] = model2.predict(X2_test_vectorized).astype(bool)
test_df["targetu"] = modelu.predict(Xu_test_vectorized).astype(bool)

test_df["target"] = test_df["target1"]
for i in range(len(test_df["target1"])):
    test_df["target"][i] = test_df["targetu"][i] or test_df["target1"][i] and test_df["target2"][i]

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)

!cat ml_baseline.csv | head

<ipython-input-297-244c2a861c0b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["target"][i] = test_df["targetu"][i] or test_df["target1"][i] and test_df["target2"][i]


id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe
